In [3]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("gamer")

2024/06/19 08:19:13 INFO mlflow.tracking.fluent: Experiment with name 'gamer' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/431548450445816159', creation_time=1718785153935, experiment_id='431548450445816159', last_update_time=1718785153935, lifecycle_stage='active', name='gamer', tags={}>

In [4]:
import subprocess
import mlflow
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

train_data_path = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
valid_data_path = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"

# Load the data
df_train = pd.read_parquet(train_data_path)
df_valid = pd.read_parquet(valid_data_path)

# Feature engineering (example features)
df_train['duration'] = (df_train['tpep_dropoff_datetime'] - df_train['tpep_pickup_datetime']).dt.total_seconds() / 60
df_valid['duration'] = (df_valid['tpep_dropoff_datetime'] - df_valid['tpep_pickup_datetime']).dt.total_seconds() / 60

# Drop rows with duration outliers
df_train = df_train[(df_train['duration'] >= 1) & (df_train['duration'] <= 60)]
df_valid = df_valid[(df_valid['duration'] >= 1) & (df_valid['duration'] <= 60)]

# Example target and feature columns
target_col = 'duration'
feature_cols = ['passenger_count', 'trip_distance']

# Select the features and target
X_train = df_train[feature_cols]
y_train = df_train[target_col]
X_val = df_valid[feature_cols]
y_val = df_valid[target_col]

# Handle missing values by filling them with the mean of the column
X_train = X_train.fillna(X_train.mean())
X_val = X_val.fillna(X_val.mean())

alpha = 0.01

# Run MLflow experiment
with mlflow.start_run():
    mlflow.set_tag("developer", "sandeep")
    mlflow.log_param("train-data-path", train_data_path)
    mlflow.log_param("valid-data-path", valid_data_path)
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

# Output message
print("MLflow run completed. RMSE:", rmse)

MLflow run completed. RMSE: 10.063080570603425


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
